In [ ]:
# dont_test
# default_exp scripts
%load_ext nb_black

<IPython.core.display.Javascript object>

# Workaround for lib_name != lib_path

The script nbdev_build_docs fails trying to import from lib_name which is django_fileresponse while the correct package to import from is lib_path==fileresponse. To make nbdev_build_docs work, we create a nbdev_build_docs_patched script which replaces lib_name in settings with `fileresponse`.

In [ ]:
# export

from pathlib import Path
from configparser import ConfigParser


def get_config(settings_path):
    config = ConfigParser()
    config.read(settings_path)
    return config


def patch_config(settings_path, lib_name):
    config = get_config(settings_path)
    config["DEFAULT"]["lib_name"] = lib_name
    with settings_path.open("w") as settings_file:
        config.write(settings_file)
    return config

<IPython.core.display.Javascript object>

In [ ]:
# Test helper functions


def setup_settings_file(settings_path):
    original_lib_name = "django_fileresponse"
    settings_content = f"""
    [DEFAULT]
    lib_name = {original_lib_name}
    """
    with settings_path.open("w") as f:
        f.write(settings_content)
    return settings_path, original_lib_name


def get_actual_lib_name(settings_path):
    config = get_config(settings_path)
    return config["DEFAULT"]["lib_name"]

<IPython.core.display.Javascript object>

In [ ]:
# hide
# Test for patch_config
# arange
settings_path, original_lib_name = setup_settings_file(Path.cwd() / "test.ini")

# act
patched_lib_name = "fileresponse"
patch_config(settings_path, patched_lib_name)

# assert
actual_lib_name = get_actual_lib_name(settings_path)
assert patched_lib_name == actual_lib_name

<IPython.core.display.Javascript object>

## Contextmanager patching settings.ini lib_name

In [ ]:
# export

from contextlib import contextmanager


@contextmanager
def patch_lib_name(settings_path, lib_name):
    config = get_config(settings_path)
    original_lib_name = config["DEFAULT"]["lib_name"]
    config = patch_config(settings_path, lib_name)
    try:
        yield config
    finally:
        patch_config(settings_path, original_lib_name)

<IPython.core.display.Javascript object>

### Usage

In [ ]:
settings_path, original_lib_name = setup_settings_file(Path.cwd() / "test.ini")
with patch_lib_name(settings_path, "fileresponse") as patched_config:
    patched_lib_name = get_actual_lib_name(settings_path)
    print("lib_name original vs patched: ", original_lib_name, patched_lib_name)
reset_lib_name = get_actual_lib_name(settings_path)
print("after leaving contextmanager, lib_name is back to original: ", reset_lib_name)

lib_name original vs patched:  django_fileresponse fileresponse
after leaving contextmanager, lib_name is back to original:  django_fileresponse


<IPython.core.display.Javascript object>

In [ ]:
# Test for patch_lib_name contextmanager
# arange
settings_path, original_lib_name = setup_settings_file(Path.cwd() / "test.ini")

# act and assert
patched_lib_name = "fileresponse"
with patch_lib_name(settings_path, patched_lib_name) as patched_config:
    actual_lib_name = get_actual_lib_name(settings_path)
    assert patched_lib_name == actual_lib_name

# assert
actual_lib_name = get_actual_lib_name(settings_path)
assert original_lib_name == actual_lib_name

<IPython.core.display.Javascript object>

In [ ]:
# export

import re
import sys

from nbdev.export2html import nbdev_build_docs


def nbdev_build_docs_patched():
    project_root = Path(__file__).parent.parent
    settings_path = project_root / "settings.ini"
    with patch_lib_name(settings_path, "fileresponse"):
        sys.argv[0] = re.sub(r"(-script\.pyw|\.exe)?$", "", sys.argv[0])
        sys.exit(nbdev_build_docs())

<IPython.core.display.Javascript object>